Rusty Bargain used car sales service is developing an app to attract new customers. In that app, you can quickly find out the market value of your car. You have access to historical data: technical specifications, trim versions, and prices. You need to build the model to determine the value. 

Rusty Bargain is interested in:

- the quality of the prediction;
- the speed of the prediction;
- the time required for training

## Data preparation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from lightgbm import LGBMRegressor
import catboost

In [2]:
data = pd.read_csv('/datasets/car_data.csv')
display(data.head())
print(data.describe())

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


               Price  RegistrationYear          Power        Mileage  \
count  354369.000000     354369.000000  354369.000000  354369.000000   
mean     4416.656776       2004.234448     110.094337  128211.172535   
std      4514.158514         90.227958     189.850405   37905.341530   
min         0.000000       1000.000000       0.000000    5000.000000   
25%      1050.000000       1999.000000      69.000000  125000.000000   
50%      2700.000000       2003.000000     105.000000  150000.000000   
75%      6400.000000       2008.000000     143.000000  150000.000000   
max     20000.000000       9999.000000   20000.000000  150000.000000   

       RegistrationMonth  NumberOfPictures     PostalCode  
count      354369.000000          354369.0  354369.000000  
mean            5.714645               0.0   50508.689087  
std             3.726421               0.0   25783.096248  
min             0.000000               0.0    1067.000000  
25%             3.000000               0.0   30165.

In [3]:
print(data.groupby('RegistrationYear')['RegistrationYear'].count())
data_years = data['RegistrationYear'].isin(range(1950,2020))
data_year = data[data_years]
display(data_year.head())

RegistrationYear
1000    37
1001     1
1039     1
1111     3
1200     1
        ..
9000     3
9229     1
9450     1
9996     1
9999    26
Name: RegistrationYear, Length: 151, dtype: int64


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


Some of the years in the registration years are nonsenical. I narrowed the registration year down from 1950 to 2020. 

In [4]:
print(data_year.groupby('RegistrationMonth')['RegistrationMonth'].count())
data_new = data_year[data_year['RegistrationMonth'] != 0].drop('NumberOfPictures', axis = 1)
display(data_new.head())
print(data_new.shape)

RegistrationMonth
0     37127
1     23181
2     21254
3     34363
4     29260
5     29143
6     31495
7     27196
8     22624
9     23809
10    26097
11    24184
12    24285
Name: RegistrationMonth, dtype: int64


,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,PostalCode,LastSeen
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,60437,06/04/2016 10:17
5,04/04/2016 17:36,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,04/04/2016 00:00,33775,06/04/2016 19:17


(316891, 15)


I dropped the values that had registration month of 0, and all of the values in Number of pictures were 0, so I dropped the whole column. 

In [5]:
print(data_new.info())
print(data_new.groupby('NotRepaired')['NotRepaired'].count())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 316891 entries, 1 to 354368
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        316891 non-null  object
 1   Price              316891 non-null  int64 
 2   VehicleType        294368 non-null  object
 3   RegistrationYear   316891 non-null  int64 
 4   Gearbox            306881 non-null  object
 5   Power              316891 non-null  int64 
 6   Model              303785 non-null  object
 7   Mileage            316891 non-null  int64 
 8   RegistrationMonth  316891 non-null  int64 
 9   FuelType           297682 non-null  object
 10  Brand              316891 non-null  object
 11  NotRepaired        267902 non-null  object
 12  DateCreated        316891 non-null  object
 13  PostalCode         316891 non-null  int64 
 14  LastSeen           316891 non-null  object
dtypes: int64(6), object(9)
memory usage: 38.7+ MB
None
NotRepaired
no   

In [6]:
print(data_year.groupby('VehicleType')['VehicleType'].count())
print(data_year.groupby('Gearbox')['Gearbox'].count())
print(data_year.groupby('Model')['Model'].count())

VehicleType
bus            28772
convertible    20191
coupe          16158
other           3266
sedan          91430
small          79813
suv            11981
wagon          65162
Name: VehicleType, dtype: int64
Gearbox
auto       66274
manual    268146
Name: Gearbox, dtype: int64
Model
100         416
145          49
147         572
156         627
159         212
           ... 
yaris      1033
yeti        174
ypsilon     209
z_reihe     717
zafira     3040
Name: Model, Length: 250, dtype: int64


In [7]:
data_dropped = data_new.dropna()
print(data_dropped.info())
data_filled = data_new.fillna('Na')
print(data_filled.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 238263 entries, 3 to 354367
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        238263 non-null  object
 1   Price              238263 non-null  int64 
 2   VehicleType        238263 non-null  object
 3   RegistrationYear   238263 non-null  int64 
 4   Gearbox            238263 non-null  object
 5   Power              238263 non-null  int64 
 6   Model              238263 non-null  object
 7   Mileage            238263 non-null  int64 
 8   RegistrationMonth  238263 non-null  int64 
 9   FuelType           238263 non-null  object
 10  Brand              238263 non-null  object
 11  NotRepaired        238263 non-null  object
 12  DateCreated        238263 non-null  object
 13  PostalCode         238263 non-null  int64 
 14  LastSeen           238263 non-null  object
dtypes: int64(6), object(9)
memory usage: 29.1+ MB
None
<class 'pandas.co

I noticed the columns with integers were filled out, in order to keep the most amount of data for the models I filled the null values in the categorical columns with a filler ('Na'). 

## Feature preparation

In [8]:
columns_obj = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
data_training, data_test = train_test_split(data_filled, test_size = 0.25, random_state = 12345)
data_train, data_valid = train_test_split(data_training, test_size = 0.33, random_state = 12345)

data_train_f = data_train.drop(['Price', 'DateCrawled','DateCreated', 'LastSeen'] , axis = 1)
data_valid_f = data_valid.drop(['Price', 'DateCrawled','DateCreated', 'LastSeen'] , axis = 1)
data_train_t = data_train['Price']
data_valid_t = data_valid['Price']
data_test_f = data_test.drop(['Price', 'DateCrawled','DateCreated', 'LastSeen'], axis = 1)
data_test_t = data_test['Price']


print(data_train_f.shape)
print(data_train_t.shape)
print(data_valid_f.shape)
print(data_valid_t.shape)

print(data_test_f.shape)
print(data_test_t.shape)

(159237, 11)
(159237,)
(78431, 11)
(78431,)
(79223, 11)
(79223,)


### Ordinal Encoder

In [9]:
data_train_obj = data_train_f[columns_obj]
encoder = OrdinalEncoder()
encoder.fit(data_train_obj)
train_ordinal = pd.DataFrame(encoder.transform(data_train_obj), columns=data_train_obj.columns)

data_valid_obj = data_valid_f[columns_obj]
encoder.fit(data_valid_obj)
valid_ordinal = pd.DataFrame(encoder.transform(data_valid_obj), columns=data_valid_obj.columns)

data_test_obj = data_test_f[columns_obj]
encoder.fit(data_test_obj)
test_ordinal = pd.DataFrame(encoder.transform(data_test_obj), columns=data_test_obj.columns)
display(train_ordinal.head())
display(valid_ordinal.head())
display(test_ordinal.head())

data_train_num = data_train_f.drop(columns_obj, axis = 1).reset_index(drop = True)
data_train_ord = pd.concat([data_train_num, train_ordinal], axis = 1)
display(data_train_ord.head())

data_valid_num = data_valid_f.drop(columns_obj, axis = 1).reset_index(drop = True)
data_valid_ord = pd.concat([data_valid_num, valid_ordinal], axis = 1)
display(data_valid_ord.head())

data_test_num = data_test_f.drop(columns_obj, axis = 1).reset_index(drop = True)
data_test_ord = pd.concat([data_test_num, test_ordinal], axis = 1)
display(data_test_ord.head())


,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,3.0,2.0,58.0,7.0,5.0,1.0
1,6.0,2.0,88.0,7.0,8.0,0.0
2,3.0,1.0,75.0,7.0,20.0,1.0
3,6.0,2.0,76.0,7.0,27.0,1.0
4,2.0,2.0,208.0,7.0,20.0,1.0


,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,2.0,2.0,11.0,7.0,2.0,1.0
1,8.0,2.0,154.0,7.0,10.0,1.0
2,5.0,2.0,11.0,3.0,2.0,1.0
3,7.0,2.0,180.0,7.0,23.0,1.0
4,5.0,2.0,43.0,7.0,24.0,1.0


,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,1.0,2.0,237.0,3.0,24.0,1.0
1,8.0,1.0,233.0,7.0,24.0,0.0
2,5.0,2.0,20.0,0.0,1.0,0.0
3,8.0,2.0,117.0,3.0,38.0,0.0
4,6.0,2.0,200.0,7.0,9.0,1.0


,RegistrationYear,Power,Mileage,RegistrationMonth,PostalCode,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,2005,109,90000,11,44265,3.0,2.0,58.0,7.0,5.0,1.0
1,1999,55,100000,2,50226,6.0,2.0,88.0,7.0,8.0,0.0
2,2000,306,150000,1,13355,3.0,1.0,75.0,7.0,20.0,1.0
3,2007,111,125000,7,42653,6.0,2.0,76.0,7.0,27.0,1.0
4,2005,272,125000,3,76829,2.0,2.0,208.0,7.0,20.0,1.0


,RegistrationYear,Power,Mileage,RegistrationMonth,PostalCode,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,2001,170,150000,2,84149,2.0,2.0,11.0,7.0,2.0,1.0
1,1993,116,150000,4,66482,8.0,2.0,154.0,7.0,10.0,1.0
2,2006,163,150000,3,61250,5.0,2.0,11.0,3.0,2.0,1.0
3,2009,141,125000,10,51399,7.0,2.0,180.0,7.0,23.0,1.0
4,2010,101,60000,6,67346,5.0,2.0,43.0,7.0,24.0,1.0


,RegistrationYear,Power,Mileage,RegistrationMonth,PostalCode,VehicleType,Gearbox,Model,FuelType,Brand,NotRepaired
0,2012,114,150000,2,52385,1.0,2.0,237.0,3.0,24.0,1.0
1,1998,170,60000,1,47807,8.0,1.0,233.0,7.0,24.0,0.0
2,1993,116,150000,12,15344,5.0,2.0,20.0,0.0,1.0,0.0
3,2005,101,150000,11,20357,8.0,2.0,117.0,3.0,38.0,0.0
4,2000,54,150000,6,84453,6.0,2.0,200.0,7.0,9.0,1.0


### OHE

In [10]:
data_ohe = pd.get_dummies(data_filled[columns_obj], drop_first=True)
data_ohe_filled = data_filled.drop(columns_obj, axis = 1)
print(data_ohe.columns)
data_ohe_fill = pd.concat([data_ohe_filled, data_ohe], axis = 1)
print(data_ohe_filled.shape)

Index(['VehicleType_bus', 'VehicleType_convertible', 'VehicleType_coupe',
       'VehicleType_other', 'VehicleType_sedan', 'VehicleType_small',
       'VehicleType_suv', 'VehicleType_wagon', 'Gearbox_auto',
       'Gearbox_manual',
       ...
       'Brand_smart', 'Brand_sonstige_autos', 'Brand_subaru', 'Brand_suzuki',
       'Brand_toyota', 'Brand_trabant', 'Brand_volkswagen', 'Brand_volvo',
       'NotRepaired_no', 'NotRepaired_yes'],
      dtype='object', length=307)
(316891, 9)


In [11]:
data_training_ohe, data_test_ohe = train_test_split(data_ohe_fill, test_size = 0.25, random_state = 12345)
data_train_ohe, data_valid_ohe = train_test_split(data_training_ohe, test_size = 0.33, random_state = 12345)
data_train_f_ohe = data_train_ohe.drop(['Price', 'DateCrawled','DateCreated', 'LastSeen'] , axis = 1)
data_train_t_ohe = data_train_ohe['Price']
data_valid_f_ohe = data_valid_ohe.drop(['Price', 'DateCrawled','DateCreated', 'LastSeen'] , axis = 1)
data_valid_t_ohe = data_valid_ohe['Price']

data_test_f_ohe = data_test_ohe.drop(['Price', 'DateCrawled','DateCreated', 'LastSeen'], axis = 1)
data_test_t_ohe = data_test_ohe['Price']
print(data_train_f_ohe.shape)
print(data_train_t_ohe.shape)
print(data_valid_f_ohe.shape)
print(data_valid_t_ohe.shape)
print(data_test_f_ohe.shape)
print(data_test_t_ohe.shape)

(159237, 312)
(159237,)
(78431, 312)
(78431,)
(79223, 312)
(79223,)


## Model training

### Random Forest Classifier

In [12]:
best_depth = 0
best_score = 0
best_est = 0

In [13]:
%%time
best_score = 100000000
best_est = 0
best_ran_for = 0
for est in range(1, 51, 10):
    model_ran_for = RandomForestRegressor(random_state=54321, n_estimators=est) 
    model_ran_for.fit(train_ordinal, data_train_t)
    predicts = model_ran_for.predict(valid_ordinal)
    score = mean_squared_error(predicts, data_valid_t)
    if score < best_score:
        best_est = est
        best_score = score
        best_ran_for = model_ran_for 


CPU times: user 24.2 s, sys: 48.4 ms, total: 24.3 s
Wall time: 24.3 s


In [14]:
print(best_score)
print(best_est)

13110979.699184418
41


In [15]:
%%time
model_ran_for.fit(train_ordinal, data_train_t)
predicts = best_ran_for.predict(test_ordinal)

CPU times: user 9.44 s, sys: 11.7 ms, total: 9.46 s
Wall time: 9.48 s


### Linear Regression

In [16]:
%%time
lin_reg = LinearRegression()
lin_reg.fit(data_train_f_ohe, data_train_t_ohe)
lin_predict = lin_reg.predict(data_test_f_ohe)

CPU times: user 11.9 s, sys: 2.11 s, total: 14 s
Wall time: 14 s


In [17]:
lin_rmse = mean_squared_error(lin_predict, data_test_t_ohe)
print(lin_rmse)

8074072.869412064


### LGB model

In [18]:
%%time

lgb_score = 100000000
best_depth = 0
for depth in range(1, 51, 10):
    lgb_model = LGBMRegressor(random_state = 54321, max_depth = depth) 
    lgb_model.fit(data_train_f_ohe, data_train_t_ohe)
    lgb_predict = lgb_model.predict(data_valid_f_ohe)
    score = mean_squared_error(lgb_predict, data_valid_t)
    if score < best_score:
        best_depth = depth
        lgb_score = score
print(lgb_score)
print(best_depth)

3346365.2313237227
41
CPU times: user 27 s, sys: 1.05 s, total: 28 s
Wall time: 28.1 s


In [19]:
%%time
lgb_score = 100000000
best_bins = 0
for bins in range(3000, 3500, 50):
    lgb_model = LGBMRegressor(random_state = 54321, max_depth = 41, max_bins = bins) 
    lgb_model.fit(data_train_f_ohe, data_train_t_ohe)
    lgb_predict = lgb_model.predict(data_valid_f_ohe)
    score = mean_squared_error(lgb_predict, data_valid_t)
    if score < best_score:
        best_bins = bins
        lgb_score = score
print(lgb_score)
print(best_bins)

3347517.397991262
3450
CPU times: user 1min 11s, sys: 1.95 s, total: 1min 13s
Wall time: 1min 13s


In [20]:
%%time
lgb_model = LGBMRegressor(random_state = 54321, max_depth = 41, max_bins = 3450) 
lgb_model.fit(data_train_f_ohe, data_train_t_ohe)
lgb_predict = lgb_model.predict(data_test_f_ohe)

CPU times: user 7.31 s, sys: 210 ms, total: 7.52 s
Wall time: 7.58 s


### Cat Boost

In [21]:
%%time
cb_model = catboost.CatBoostRegressor()
cb_model.fit(data_train_f_ohe, data_train_t_ohe)

Learning rate set to 0.091223
0:	learn: 4324.4191702	total: 87.6ms	remaining: 1m 27s
1:	learn: 4098.9577528	total: 122ms	remaining: 1m 1s
2:	learn: 3900.0981431	total: 159ms	remaining: 53s
3:	learn: 3710.9900507	total: 194ms	remaining: 48.3s
4:	learn: 3547.6668884	total: 232ms	remaining: 46.1s
5:	learn: 3399.9174232	total: 269ms	remaining: 44.5s
6:	learn: 3264.6293451	total: 308ms	remaining: 43.7s
7:	learn: 3149.0146625	total: 345ms	remaining: 42.8s
8:	learn: 3052.3714951	total: 381ms	remaining: 42s
9:	learn: 2958.5975312	total: 418ms	remaining: 41.4s
10:	learn: 2877.9383342	total: 456ms	remaining: 41s
11:	learn: 2804.5264072	total: 491ms	remaining: 40.5s
12:	learn: 2742.5234237	total: 534ms	remaining: 40.5s
13:	learn: 2683.2418039	total: 569ms	remaining: 40.1s
14:	learn: 2633.9036636	total: 606ms	remaining: 39.8s
15:	learn: 2585.8010858	total: 638ms	remaining: 39.2s
16:	learn: 2544.3586327	total: 675ms	remaining: 39.1s
17:	learn: 2510.0381136	total: 708ms	remaining: 38.6s
18:	learn: 2

In [22]:
%%time
cb_predict = cb_model.predict(data_test_f_ohe)
cb_rmse = mean_squared_error(cb_predict, data_test_t_ohe)
print(cb_rmse)

3079130.2126403376
CPU times: user 167 ms, sys: 12 ms, total: 179 ms
Wall time: 177 ms


## Model analysis

In [23]:
random_forest = best_score / 1000000
lin_reg = lin_rmse / 1000000
lgb_model = lgb_score / 1000000
cat_boost = cb_rmse / 1000000
print('Random Forest: ', random_forest, 'x10^6')
print('Linear Regression: ', lin_reg, 'x10^6')
print('LGB_model: ', lgb_model, 'x10^6')
print('CatBoost:' , cat_boost, 'x10^6')

Random Forest:  13.110979699184417 x10^6
Linear Regression:  8.074072869412063 x10^6
LGB_model:  3.347517397991262 x10^6
CatBoost: 3.0791302126403375 x10^6


<div class="alert alert-info">
The only reason I divided the RMSE is was only because it's easier for me to compare the scores with simple numbers. I added 10^6 to avoid confusion. 
</div>

## Conclusion

The time to train and predict for the random forest model was 9.46s, linear regression was 14s, LGB model was 7.52s, and catboost was 33.3s. 

Once the hyperameters were set the LGB model had the fastest run time. However the CatBoost model had slightly lower RMSE score. Overall, I would recomend LGB to be the better model because it is the fastest model to fit and predict the data and the error is not compromised with the speed.   
